In [10]:
import folium
import re
import fileinput
import pandas as pd
from folium import IFrame

In [11]:
VEGAS_COORDINATES = (36.1699, -115.1398)
map = folium.Map(location=VEGAS_COORDINATES, zoom_start=11)

In [12]:
pos_df = pd.read_csv('pos_bsns.csv')
neg_df = pd.read_csv('neg_bsns.csv')
del(pos_df['Unnamed: 0'])
del(neg_df['Unnamed: 0'])

In [13]:
pos_topics = {0:'menu/experience', 1:'burger/fries', 2:'service/atmosphere', 
              3:'breakfast/pancakes', 4:'chicken/waffles', 5:'location/atmosphere'}

neg_topics = {0:'wait/table', 1:'burger/fries', 2:'bad_service', 
              3:'chicken/salad', 4:'breakfast/eggs', 5:'buffet/price'}

In [14]:
def star_color(stars):
    if stars < 2.0:
        return 'darkred'
    elif stars < 3.0:
        return 'lightred'
    elif stars < 4.0:
        return 'orange'
    elif stars < 5.0:
        return 'lightgreen'
    else:
        return 'darkgreen'

def html_text(topics, stars, scores, name):
    html_topics = ['','','','','','']
    for n in topics:
        html_topics[n] = 'Topic {}: {:18} {:.2f}%'.format(n, neg_topics[n], scores[n]*100)
    topics_text = '<br>'.join(html_topics)
    stars_text = '<h3> Stars: ' + str(stars) + '</h3>'
    name_text = '<h3> Name: ' + name + '</h3>'
    return name_text + stars_text + '<p>' + topics_text + '</p>'

def get_popup(topics, stars, scores, name):
    html = "" + html_text(topics, stars, scores, name) + ""
    iframe = folium.IFrame(html=html, width=300, height=200)
    return folium.Popup(iframe, max_width=1000)

In [15]:
for row in neg_df.itertuples():
    #print(row)
    probs = [row._7, row._8, row._9, row._10, row._11, row._12]
    folium.Marker(location=[row.latitude, row.longitude], popup=get_popup(neg_topics, row.stars_y, probs, row.name), 
                  icon=folium.Icon(color=star_color(row.stars_y), icon='map-pin')).add_to(map)
    if row.Index > 300:
        break

In [16]:
for row2 in neg_df.itertuples():
    #print(row)
    probs = [row2._7, row2._8, row2._9, row2._10, row2._11, row2._12]
    folium.Marker(location=[row2.latitude, row2.longitude], popup=get_popup(pos_topics, row2.stars_y, probs, row2.name), 
                  icon=folium.Icon(color=star_color(row2.stars_y), icon='map-pin')).add_to(map)
    if row.Index > 300:
        break

In [17]:
display(map)

In [18]:
map.save('vegas.html')

In [19]:
with open("vegas.html") as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for linenum,line in enumerate( fileinput.FileInput("vegas.html",inplace=1) ):
   pattern = markers[0] + ".bindPopup"
   pattern2 = markers[0] + ".on('mouseover', function (e) {this.openPopup();});"
   pattern3 = markers[0] + ".on('mouseout', function (e) {this.closePopup();});"

   if pattern in line:
      print(line.rstrip())
      print(pattern2)
      print(pattern3)
   else:
      print(line.rstrip())